# Laboratorio 8

Vamos a empezar uniendo todos los datasets. Empezamos con el de hechos. En una exploración inicial se vio que cada dataset tiene diferentes nombres para columnas con los mismos valores y que en los años recientes se han añadido más campos, vamos a ver cuales en específico cumplen con esto


Sabiendo esto vamos a unirlos con las columnas en comun. 

In [0]:
import pandas as pd

hechos_files = ["Hechos_2013.csv", 
                "Hechos_2014.csv", 
                "Hechos_2015.csv", 
                "Hechos_2016.csv",
                "Hechos_2017.csv",
                "Hechos_2018.csv",
                "Hechos_2019.csv",
                "Hechos_2020.csv",
                "Hechos_2021.csv",
                "Hechos_2022.csv",
                "Hechos_2023.csv"]
lesionados_files = ["Lesionados_2013.csv",
                    "Lesionados_2014.csv",
                    "Lesionados_2015.csv",
                    "Lesionados_2016.csv",
                    "Lesionados_2017.csv",
                    "Lesionados_2018.csv",
                    "Lesionados_2019.csv",
                    "Lesionados_2020.csv",
                    "Lesionados_2021.csv",
                    "Lesionados_2022.csv",
                    "Lesionados_2023.csv"]

vehiculos_files = [
                    "Vehículos_2013.csv",
                    "Vehículos_2014.csv",
                    "Vehículos_2015.csv",
                    "Vehículos_2016.csv",
                    "Vehículos_2017.csv",
                    "Vehículos_2018.csv",
                    "Vehículos_2019.csv",
                    "Vehículos_2020.csv",
                    "Vehículos_2021.csv",
                    "Vehículos_2022.csv",
                    "Vehículos_2023.csv"
                    ]

hechos_df = pd.concat([pd.read_csv("/Volumes/workspace/default/lab8_new/"+f) for f in hechos_files], ignore_index=True)
lesionados_df = pd.concat([pd.read_csv("/Volumes/workspace/default/lab8_new/"+f) for f in lesionados_files], ignore_index=True)
vehiculos_df = pd.concat([pd.read_csv("/Volumes/workspace/default/lab8_new/"+f) for f in vehiculos_files], ignore_index=True)

print("Filas Hechos:", len(hechos_df))
print("Filas Lesionados:", len(lesionados_df))
print("Filas Vehículos:", len(vehiculos_df))

common_cols = set(hechos_df.columns) & set(lesionados_df.columns) & set(vehiculos_df.columns)
print("Columnas comunes usadas para el merge:", common_cols)

merged_df = pd.merge(hechos_df, lesionados_df, on=list(common_cols), how="outer")
merged_df = pd.merge(merged_df, vehiculos_df, on=list(common_cols), how="outer")

output_file = "/Volumes/workspace/default/lab8_new/Datos_Combinados_2013_2023.csv"

merged_df.to_csv(output_file, index=False, encoding="utf-8")


print("Columnas finales:", merged_df.columns.tolist())
print("Filas totales después del merge:", len(merged_df))
print(f"Archivo combinado guardado como: {output_file}")


## Analisis Exploratorio

**Mostrar cuántos registros hay en cada dataset (hechos, vehículos, fallecidos,
lesionados). Muestre algunos resultados con la función .show(). Genere un describe y
summary para aquellas columnas que considere importantes según cada archivo.**

In [0]:
import pandas as pd


base_path = "/Volumes/workspace/default/lab8_new/"
# --- Función para cargar y concatenar --- #
def load_concat(files):
    return pd.concat([pd.read_csv(base_path + f) for f in files], ignore_index=True)

# --- Cargar datasets --- #
hechos_df = load_concat(hechos_files)
lesionados_df = load_concat(lesionados_files)
vehiculos_df = load_concat(vehiculos_files)

# --- Mostrar número de registros --- #
print("Registros por dataset:")
print(f"Hechos: {len(hechos_df)}")
print(f"Lesionados: {len(lesionados_df)}")
print(f"Vehículos: {len(vehiculos_df)}\n")

# --- Mostrar algunos registros --- #
print("Ejemplo de Hechos:")
print(hechos_df.head(), "\n")

print("Ejemplo de Lesionados:")
print(lesionados_df.head(), "\n")

print("Ejemplo de Vehículos:")
print(vehiculos_df.head(), "\n")

# --- Estadísticas descriptivas (describe) --- #
num_cols_hechos = hechos_df.select_dtypes(include='number').columns.tolist()
num_cols_lesionados = lesionados_df.select_dtypes(include='number').columns.tolist()
num_cols_vehiculos = vehiculos_df.select_dtypes(include='number').columns.tolist()

print("Describe Hechos:")
print(hechos_df[num_cols_hechos].describe(), "\n")

print("Describe Lesionados:")
print(lesionados_df[num_cols_lesionados].describe(), "\n")

print("Describe Vehículos:")
print(vehiculos_df[num_cols_vehiculos].describe(), "\n")

# --- Resúmenes categóricos --- #
cat_cols_hechos = ['Tipo de accidente', 'Departamento']
cat_cols_lesionados = ['Tipo de lesión', 'Sexo']

for col in cat_cols_hechos:
    if col in hechos_df.columns:
        print(f"Resumen categórico Hechos ({col}):")
        print(hechos_df[col].value_counts(), "\n")

for col in cat_cols_lesionados:
    if col in lesionados_df.columns:
        print(f"Resumen categórico Lesionados ({col}):")
        print(lesionados_df[col].value_counts(), "\n")


**Identificar los años disponibles en cada dataset y validar si coinciden.**

In [0]:
# Función para extraer años únicos de una columna de fecha o año
def extraer_anios(df, col_name):
    if col_name in df.columns:
        # Si la columna es de tipo fecha, extraemos el año
        if pd.api.types.is_datetime64_any_dtype(df[col_name]):
            return df[col_name].dt.year.unique()
        # Si la columna es numérica, asumimos que contiene el año
        elif pd.api.types.is_numeric_dtype(df[col_name]):
            return df[col_name].unique()
        # Si es texto, intentamos convertir a datetime
        else:
            try:
                return pd.to_datetime(df[col_name], errors='coerce').dt.year.dropna().unique()
            except:
                return []
    else:
        return []

fecha_col_hechos = 'Fecha'      # columna con fecha en hechos
fecha_col_lesionados = 'Fecha'  # columna con fecha en lesionados
fecha_col_vehiculos = 'Fecha'   # columna con fecha en vehículos

anios_hechos = sorted(extraer_anios(hechos_df, fecha_col_hechos))
anios_lesionados = sorted(extraer_anios(lesionados_df, fecha_col_lesionados))
anios_vehiculos = sorted(extraer_anios(vehiculos_df, fecha_col_vehiculos))

print("Años disponibles en Hechos:", anios_hechos)
print("Años disponibles en Lesionados:", anios_lesionados)
print("Años disponibles en Vehículos:", anios_vehiculos)

if anios_hechos == anios_lesionados == anios_vehiculos:
    print("\n✅ Todos los datasets tienen los mismos años disponibles.")
else:
    print("\n⚠️ Los datasets tienen años distintos. Diferencias:")
    print("Hechos vs Lesionados:", set(anios_hechos) - set(anios_lesionados))
    print("Hechos vs Vehículos:", set(anios_hechos) - set(anios_vehiculos))
    print("Lesionados vs Vehículos:", set(anios_lesionados) - set(anios_vehiculos))

Como podemos ver todos tienen los mismos años disponibles.

**Mostrar los valores distintos de tipo de accidente**

In [0]:
if 'causa_acc' in hechos_df.columns:
    tipos_accidente = hechos_df['causa_acc'].unique()
    print("Valores distintos de tipo de accidente (causa_acc):")
    print(tipos_accidente)
    
    print("\nFrecuencia de cada tipo de accidente:")
    print(hechos_df['causa_acc'].value_counts())
else:
    print("La columna 'causa_acc' no existe en hechos_df.")


podemos ver que el mas frecuente son los accidentes de tipo 5.

**Calcular cuántos departamentos únicos aparecen en las bases**

In [0]:
# --- Departamentos únicos --- #
if 'depto_ocu' in hechos_df.columns:
    deptos_hechos = hechos_df['depto_ocu'].unique()
    print(f"Hechos - departamentos únicos ({len(deptos_hechos)}):")
    print(deptos_hechos, "\n")

if 'depto_ocu' in lesionados_df.columns:
    deptos_lesionados = lesionados_df['depto_ocu'].unique()
    print(f"Lesionados - departamentos únicos ({len(deptos_lesionados)}):")
    print(deptos_lesionados, "\n")

if 'depto_ocu' in vehiculos_df.columns:
    deptos_vehiculos = vehiculos_df['depto_ocu'].unique()
    print(f"Vehículos - departamentos únicos ({len(deptos_vehiculos)}):")
    print(deptos_vehiculos, "\n")


Vemos que los vehiculos tienen 44 , lesionados tienen 44 y hechos tienen mas que son 50

## PREGUNTAS DE ANÁLISIS 

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

# Leer CSV desde el path local
df = spark.read.option("header", "true").option("inferSchema", "true") \
    .csv("/Volumes/workspace/default/lab8_new/Datos_Combinados_2013_2023.csv")

# Mostrar columnas
print("Columnas del DataFrame:")
print(df.columns)

# Mostrar esquema completo
df.printSchema()


**¿Cuál es el total de accidentes por año y departamento? Apóyese de la función
groupBy. Investigue la función display que tiene Databricks y muestre su resultado en
formato de gráfico de barras**

In [0]:
from pyspark.sql.functions import col

# Seleccionamos solo las columnas que necesitamos: año y departamento
df_sel = df.select(
    col("año_ocu").alias("anio"),
    col("depto_ocu").alias("departamento")
)

# Filtrar filas donde año o departamento sean nulos
df_clean = df_sel.dropna(subset=["anio", "departamento"])

accidentes_anio_depto = df_clean.groupBy("anio", "departamento").count().orderBy("anio", "departamento")

display(accidentes_anio_depto)  # En Databricks, display permite elegir gráfico de barras desde la UI

In [0]:
import pandas as pd
import matplotlib.pyplot as plt

# Convertir a Pandas
df_pandas = accidentes_anio_depto.toPandas()

# Crear pivot para mejor visualización
df_pivot = df_pandas.pivot(index='departamento', columns='anio', values='count').fillna(0)

# Crear gráfico de barras
df_pivot.plot(kind='bar', figsize=(15, 6), width=0.8)
plt.title('Accidentes por Departamento y Año')
plt.xlabel('Departamento')
plt.ylabel('Cantidad de Accidentes')
plt.legend(title='Año')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

Podemos ver que el que mas tiene es el de Guatemala, seguido por el de Escuintla.

**¿Qué día de la semana registra más accidentes en 2023? Graficar con display en un
gráfico de columnas.**

In [0]:
from pyspark.sql.functions import col, count
import matplotlib.pyplot as plt

# Filtrar solo datos de 2023 y eliminar nulos en día_sem_ocu
df_2023 = df.filter((col("año_ocu") == 2023) & (col("día_sem_ocu").isNotNull()))

# Agrupar por día de la semana y contar accidentes
accidentes_por_dia = df_2023.groupBy("día_sem_ocu").count().orderBy("día_sem_ocu")

# Mostrar la tabla
display(accidentes_por_dia)


In [0]:
import matplotlib.pyplot as plt

# Convertir a Pandas para graficar
df_pandas = accidentes_por_dia.toPandas()

# Crear gráfico de columnas
fig, ax = plt.subplots(figsize=(12, 6))
bars = ax.bar(df_pandas['día_sem_ocu'], df_pandas['count'], color='steelblue', edgecolor='black', linewidth=1.5)

ax.set_title('Accidentes por Día de la Semana en 2023', fontsize=16, fontweight='bold', pad=20)
ax.set_xlabel('Día de la Semana', fontsize=12)
ax.set_ylabel('Cantidad de Accidentes', fontsize=12)
ax.tick_params(axis='x', rotation=45)
ax.grid(axis='y', alpha=0.3, linestyle='--')

# Agregar valores sobre las barras
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}',
            ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.subplots_adjust(bottom=0.2)
plt.show()

# Mostrar el día con más accidentes
dia_max_idx = df_pandas['count'].idxmax()
dia_max = df_pandas.iloc[dia_max_idx]
print(f"\n📊 Día con más accidentes en 2023: {dia_max['día_sem_ocu']} con {int(dia_max['count'])} accidentes")

Vemos que el dia con mas accidentes es Domingo , posiblemente porque en este dia la gente sale mas y como es fin de semana tiene menor precaucion.


**Mostrar la distribución de accidentes por hora del día en el municipio de
Guatemala. Graficar en un histograma**


In [0]:


# Filtrar datos del municipio de Guatemala
# Primero verificar el nombre exacto del municipio
print("Municipios únicos:")
df.select("mupio_ocu").distinct().show(50, truncate=False)

# Filtrar por municipio de Guatemala y eliminar nulos en hora_ocu
df_guatemala = df.filter(
    (col("mupio_ocu") == "Guatemala") & 
    (col("hora_ocu").isNotNull())
)

# Seleccionar solo la hora
accidentes_por_hora = df_guatemala.select("hora_ocu")

# Mostrar tabla agrupada
display(df_guatemala.groupBy("hora_ocu").count().orderBy("hora_ocu"))

# Convertir a Pandas
df_pandas = accidentes_por_hora.toPandas()

# Convertir hora_ocu a numérico (puede estar en formato HH:MM o solo hora)
# Extraer solo la hora si está en formato HH:MM
df_pandas['hora'] = df_pandas['hora_ocu'].astype(str).str.split(':').str[0]
df_pandas['hora'] = pd.to_numeric(df_pandas['hora'], errors='coerce')

# Eliminar valores nulos o inválidos
df_pandas = df_pandas.dropna(subset=['hora'])
df_pandas = df_pandas[(df_pandas['hora'] >= 0) & (df_pandas['hora'] <= 23)]

# Crear histograma
fig, ax = plt.subplots(figsize=(14, 6))
n, bins, patches = ax.hist(df_pandas['hora'], bins=24, range=(0, 24), 
                           color='coral', edgecolor='black', linewidth=1.2, alpha=0.7)

ax.set_title('Distribución de Accidentes por Hora del Día en Guatemala', 
             fontsize=16, fontweight='bold', pad=20)
ax.set_xlabel('Hora del Día (0-23)', fontsize=12)
ax.set_ylabel('Cantidad de Accidentes', fontsize=12)
ax.set_xticks(range(0, 24))
ax.grid(axis='y', alpha=0.3, linestyle='--')

# Agregar valores sobre las barras
for i in range(len(n)):
    if n[i] > 0:
        ax.text(bins[i] + 0.5, n[i], f'{int(n[i])}', 
                ha='center', va='bottom', fontsize=8)

plt.subplots_adjust(bottom=0.1)
plt.show()

# Estadísticas
print(f"\n📊 Total de accidentes en Guatemala: {len(df_pandas)}")
print(f"🕐 Hora con más accidentes: {int(df_pandas['hora'].mode()[0])}:00 hrs")

Podemos ver que en Guatemala a la hora con mas accidentes es en la noche. Muy posiblemente porque al ser una hora muy tarde se tiene poca visibilidad.

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
import unicodedata

spark = SparkSession.builder.getOrCreate()

def normalize_col(colname: str) -> str:
    colname = colname.strip().lower()
    colname = unicodedata.normalize('NFKD', colname).encode('ascii', 'ignore').decode('utf-8')
    return colname

hechos_renames = {
    "num_hecho": "num_corre", "num_correlativo": "num_corre", "num_corre": "num_corre", "num": "num_corre", "nro_hecho": "num_corre",
    "dia_ocu": "dia_ocu", "dia": "dia_ocu", "dia_ocurrencia": "dia_ocu",
    "mes_ocu": "mes_ocu", "mes": "mes_ocu",
    "anio_ocu": "anio_ocu", "ano_ocu": "anio_ocu", "año_ocu": "anio_ocu",
    "hora_ocu": "hora_ocu", "hora": "hora_ocu",
    "g_hora": "g_hora", "g_hora_5": "g_hora_5",
    "areag_ocu": "area_geo_ocu", "area_geo_ocu": "area_geo_ocu",
    "depto_ocu": "depto_ocu", "mupio_ocu": "mupio_ocu", "zona_ocu": "zona_ocu",
    "sexo_pil": "sexo", "sexo_con": "sexo", "sexo_per": "sexo",
    "edad_pil": "edad", "edad_con": "edad", "edad_per": "edad",
    "estado_pil": "estado_con", "estado_con": "estado_con",
    "marca_veh": "marca_veh", "tipo_veh": "tipo_veh", "color_veh": "color_veh", "modelo_veh": "modelo_veh",
    "g_modelo_veh": "g_modelo_veh", "tipo_eve": "tipo_eve"
}

vehiculos_renames = hechos_renames 

hechos_final_cols = [
    "num_corre", "anio_ocu", "mes_ocu", "dia_ocu", "dia_sem_ocu", "hora_ocu", "g_hora", "g_hora_5",
    "depto_ocu", "mupio_ocu", "area_geo_ocu", "zona_ocu", "sexo", "edad", "mayor_menor",
    "estado_con", "tipo_veh", "marca_veh", "color_veh", "modelo_veh", "g_modelo_veh", "tipo_eve", "causa_acc"
]

vehiculos_final_cols = hechos_final_cols[:-1]  

def normalizar_df(path, year, renames, final_cols):
    df = (spark.read
        .option("header", "true")
        .option("inferSchema", "true")
        .csv("/Volumes/workspace/default/lab8_new/" + path))

    df = df.toDF(*[normalize_col(c) for c in df.columns])

    for old, new in renames.items():
        if old in df.columns:
            df = df.withColumnRenamed(old, new)


    if "anio_ocu" not in df.columns:
        df = df.withColumn("anio_ocu", lit(year))

    for c in final_cols:
        if c not in df.columns:
            df = df.withColumn(c, lit(None))
    df = df.select(final_cols)
    return df

hechos_dfs = [normalizar_df(f, year, hechos_renames, hechos_final_cols) for f, year in zip(hechos_files, range(2013, 2024))]
hechos_general = hechos_dfs[0]
for df in hechos_dfs[1:]:
    hechos_general = hechos_general.unionByName(df, allowMissingColumns=True)

vehiculos_dfs = [normalizar_df(f, year, vehiculos_renames, vehiculos_final_cols) for f, year in zip(vehiculos_files, range(2013, 2024))]
vehiculos_general = vehiculos_dfs[0]
for df in vehiculos_dfs[1:]:
    vehiculos_general = vehiculos_general.unionByName(df, allowMissingColumns=True)


Vamos a usar el siguiente criterio para unir las tablas:

- Que tengan el mismo mes, semana, día, año, hora, semana, etc.
- Que tengan el mismo valor en todo lo relacionado al vehículo (marca, tipo, modelo, etc. )
- Que hayan ocurrido en el mismo lugar y que las personas involucradas tengan las mismas características. 

In [0]:

from pyspark.sql.functions import col
vehiculos_clean = vehiculos_general.filter(
    col("anio_ocu").isNotNull() &
    col("mes_ocu").isNotNull() &
    col("dia_sem_ocu").isNotNull() &
    col("dia_ocu").isNotNull() &
    col("hora_ocu").isNotNull() &
    col("depto_ocu").isNotNull() &
    col("mupio_ocu").isNotNull() &
    col("area_geo_ocu").isNotNull() &
    col("zona_ocu").isNotNull() &
    col("sexo").isNotNull() &
    col("edad").isNotNull() &
    col("estado_con").isNotNull() &
    col("tipo_veh").isNotNull() &
    col("marca_veh").isNotNull() &
    col("color_veh").isNotNull() &
    col("modelo_veh").isNotNull() &
    col("g_modelo_veh").isNotNull() &
    col("tipo_eve").isNotNull() &
    col("g_hora").isNotNull() &
    col("g_hora_5").isNotNull()
)

hechos_clean = hechos_general.filter(
    col("anio_ocu").isNotNull() &
    col("mes_ocu").isNotNull() &
    col("dia_sem_ocu").isNotNull() &
    col("dia_ocu").isNotNull() &
    col("hora_ocu").isNotNull() &
    col("depto_ocu").isNotNull() &
    col("mupio_ocu").isNotNull() &
    col("area_geo_ocu").isNotNull() &
    col("zona_ocu").isNotNull() &
    col("sexo").isNotNull() &
    col("edad").isNotNull() &
    col("estado_con").isNotNull() &
    col("tipo_veh").isNotNull() &
    col("marca_veh").isNotNull() &
    col("color_veh").isNotNull() &
    col("modelo_veh").isNotNull() &
    col("g_modelo_veh").isNotNull() &
    col("tipo_eve").isNotNull() &    
    col("g_hora").isNotNull() &
    col("g_hora_5").isNotNull()
)


vehiculos_clean.createOrReplaceTempView("vehiculos")
hechos_clean.createOrReplaceTempView("hecho")

In [0]:


joined = spark.sql("""SELECT h.num_corre as id_hecho, v.*,  h.causa_acc, h.mayor_menor, h.estado_con  FROM  vehiculos v join hecho h ON v.anio_ocu = h.anio_ocu AND v.mes_ocu = h.mes_ocu 
                   AND v.dia_sem_ocu = h.dia_sem_ocu AND v.dia_ocu = h.dia_ocu AND v.hora_ocu = h.hora_ocu 
                   AND v.depto_ocu = h.depto_ocu AND v.mupio_ocu = h.mupio_ocu AND v.area_geo_ocu = h.area_geo_ocu 
                   AND v.zona_ocu = h.zona_ocu AND v.sexo = h.sexo AND v.edad = h.edad AND v.estado_con = h.estado_con 
                   AND v.tipo_veh = h.tipo_veh AND v.marca_veh = h.marca_veh AND v.color_veh = h.color_veh 
                   AND v.modelo_veh = h.modelo_veh AND v.g_modelo_veh = h.g_modelo_veh AND v.tipo_eve = h.tipo_eve
                   AND v.g_hora = h.g_hora AND v.g_hora_5 = h.g_hora_5""")


In [0]:
joined.count()

Combinando las tablas y filtrando para aquellas que no tienen nulls, obtuvimos 6867 registros

In [0]:
joined.createOrReplaceTempView("accidentes_vehiculos")

Ahora vamos a ver cuántos vehículos están involucrados en cada accidente en promedio por cada departamento

In [0]:
import matplotlib.pyplot as plt
df_departamentos = spark.sql("""
SELECT 
  CASE departamento
    WHEN 1 THEN 'Guatemala'
    WHEN 2 THEN 'El Progreso'
    WHEN 3 THEN 'Sacatepéquez'
    WHEN 4 THEN 'Chimaltenango'
    WHEN 5 THEN 'Escuintla'
    WHEN 6 THEN 'Santa Rosa'
    WHEN 7 THEN 'Sololá'
    WHEN 8 THEN 'Totonicapán'
    WHEN 9 THEN 'Quetzaltenango'
    WHEN 10 THEN 'Suchitepéquez'
    WHEN 11 THEN 'Retalhuleu'
    WHEN 12 THEN 'San Marcos'
    WHEN 13 THEN 'Huehuetenango'
    WHEN 14 THEN 'Quiché'
    WHEN 15 THEN 'Baja Verapaz'
    WHEN 16 THEN 'Alta Verapaz'
    WHEN 17 THEN 'Petén'
    WHEN 18 THEN 'Izabal'
    WHEN 19 THEN 'Zacapa'
    WHEN 20 THEN 'Chiquimula'
    WHEN 21 THEN 'Jalapa'
    WHEN 22 THEN 'Jutiapa'
    ELSE 'Desconocido'
  END AS nombre_departamento,
  AVG(total_autos) AS promedio_autos
FROM (
  SELECT 
    id_hecho, 
    COUNT(*) AS total_autos, 
    MAX(depto_ocu) AS departamento
  FROM accidentes_vehiculos
  GROUP BY id_hecho
)
GROUP BY departamento
ORDER BY departamento
""")

df_departamentos.write.mode("overwrite").parquet("/Volumes/workspace/default/lab8_new/departamentos_promedio_autos.parquet")
df_departamentos_pd = df_departamentos.toPandas()
figure, ax = plt.subplots(figsize=(10, 6))
ax.bar(df_departamentos_pd["nombre_departamento"], df_departamentos_pd["promedio_autos"])
ax.set_title("Promedio de vehículos involucrados en accidente en promedio por departamento ")
ax.set_xlabel("Departamento")
ax.set_ylabel("Promedio de vehículos")
ax.tick_params(axis='x', rotation=45)
ax.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()



Ahora vamos a hallar el top 5 de colores de vehículos más frecuente en accidentes

In [0]:
display(spark.sql("""
SELECT
    CASE color_veh
        WHEN 1 THEN 'Rojo'
        WHEN 2 THEN 'Blanco'
        WHEN 3 THEN 'Azul'
        WHEN 4 THEN 'Gris'
        WHEN 5 THEN 'Negro'
        WHEN 6 THEN 'Verde'
        WHEN 7 THEN 'Amarillo'
        WHEN 8 THEN 'Celeste'
        WHEN 9 THEN 'Corinto'
        WHEN 10 THEN 'Café'
        WHEN 11 THEN 'Beige'
        WHEN 12 THEN 'Turquesa'
        WHEN 13 THEN 'Marfil'
        WHEN 14 THEN 'Anaranjado'
        WHEN 15 THEN 'Morado'
        WHEN 16 THEN 'Rosado'
        WHEN 17 THEN 'Varios colores'
        WHEN 99 THEN 'Ignorado'
        ELSE 'Desconocido'
    END AS color_nombre, COUNT(*) as Total
FROM vehiculos GROUP BY color_nombre ORDER BY COUNT(*) DESC LIMIT 5;
                  """))

Vemos que en la mayoría de los accidentes se desconoce el color del vehículo, lugo vemos que los carros blancos son los que están más involucrados en accidentes segudos del color rojo, es llamativo que estos colores estén presentes pues se saba que son colores bastante atractivos y a veces pueden ser factores de distracción. 

Ahora debemos calcular cuantos lesionados hubo por atropello por mes, para eso ahora vamos a usar los datasets de lesionados y haremos las mismas tranformaciones que hicimos para vehículos y hechos

In [0]:
from pyspark.sql.functions import col, when
lesionados_renames = {
    "num_hecho": "num_corre",
    "num_correlativo": "num_corre",
    "num_corre": "num_corre",
    "Num_corre": "num_corre",
    "año_ocu": "anio_ocu",
    "ano_ocu": "anio_ocu",
    "anio_ocu": "anio_ocu",
    "dia_ocu": "dia_ocu",
    "dia": "dia_ocu",
    "dia_sem_ocu": "dia_sem_ocu",
    "mes_ocu": "mes_ocu",
    "mes": "mes_ocu",
    "hora_ocu": "hora_ocu",
    "hora": "hora_ocu",
    "g_hora": "g_hora",
    "g_hora_5": "g_hora_5",
    "depto_ocu": "depto_ocu",
    "mupio_ocu": "mupio_ocu",
    "zona_ocu": "zona_ocu",
    "zona_ciudad": "zona_ciudad",
    "areag_ocu": "area_geo_ocu",
    "area_geo_ocu": "area_geo_ocu",
    "sexo_pil": "sexo",
    "sexo_per": "sexo",
    "sexo_vic": "sexo",
    "edad_pil": "edad",
    "edad_per": "edad",
    "edad_vic": "edad",
    "g_edad": "g_edad",
    "g_edad_2": "g_edad_2",
    "edad_quinquenales": "edad_quinquenales",
    "g_edad_60ymas": "g_edad_60ymas",
    "g_edad_80ymas": "g_edad_80ymas",
    "mayor_menor": "mayor_menor",
    "Otro_g_edad_fall_les": "otro_g_edad_fall_les",
    "tipo_veh": "tipo_veh",
    "marca_veh": "marca_veh",
    "color_veh": "color_veh",
    "modelo_veh": "modelo_veh",
    "g_modelo_veh": "g_modelo_veh",
    "tipo_eve": "tipo_eve",
    "fall_les": "fall_les",
    "Fallecidos_Lesionados": "fall_les",
    "int_o_noint": "int_o_noint",
    "g_edad_80ymas": "g_edad_80ymas",
    "g_edad_60ymas": "g_edad_60ymas"
}

lesionados_final_cols = [
    "num_corre",          
    "anio_ocu",          
    "mes_ocu",            
    "dia_ocu",           
    "dia_sem_ocu",        
    "hora_ocu",           
    "g_hora",            
    "g_hora_5",           
    "depto_ocu",          
    "mupio_ocu",         
    "area_geo_ocu",       
    "zona_ocu",           
    "zona_ciudad",        
    "sexo",               
    "edad",               
    "g_edad",   
    "mayor_menor",        
    "tipo_veh",           
    "marca_veh",        
    "color_veh",          
    "modelo_veh",         
    "g_modelo_veh",       
    "tipo_eve",           
    "fall_les",           
    "int_o_noint",        
    "otro_g_edad_fall_les",
    "tipo_eve_cod"
]


lesionados_dfs = []
for f, year in zip(lesionados_files, range(2013, 2024)):
    df = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv("/Volumes/workspace/default/lab8_new/" + f)
    )
    if year == 2014:
        for c in ["num_corre", "corre_base"]:
            if c in df.columns:
                df = df.drop(c)
    df = normalizar_df(f, year, lesionados_renames, lesionados_final_cols)
    lesionados_dfs.append(df)



lesionados_general = lesionados_dfs[0]
for df in lesionados_dfs[1:]:
    lesionados_general = lesionados_general.unionByName(df, allowMissingColumns=True)



Ahora vamos a ver cuantos lesionados por atropello hubo en 2023

In [0]:
lesionados_general.createOrReplaceTempView("lesionados")

In [0]:
lesionados_general.printSchema()

In [0]:
display(spark.sql("SELECT anio_ocu, mes_ocu FROM lesionados WHERE tipo_eve_cod = 5 "))

Ahora veremos en que franja horaria ocurren más accidentes

In [0]:
spark.sql("""
SELECT 
    
    CASE 
        WHEN g_hora = 1 THEN '00:00 a 05:59'
        WHEN g_hora = 2 THEN '06:00 a 11:59'
        WHEN g_hora = 3 THEN '12:00 a 17:59'
        WHEN g_hora = 4 THEN '18:00 a 23:59'
        WHEN g_hora = 5 THEN 'Ignorada'
        ELSE 'Desconocido'
    END AS rango_hora, 
    COUNT(*) as total
FROM hecho GROUP BY rango_hora
""").show()


**Calcular, para el municipio de Guatemala, cuántos accidentes hay por zona y
cuántos fallecidos se reportan en cada una. Generar un gráfico de barras con ambos
indicadores**


In [0]:
from pyspark.sql.functions import col, count

df_resumen = df.filter(col("mupio_ocu") == "Guatemala") \
               .groupBy("zona_ocu") \
               .agg(count("*").alias("total_accidentes")) \
               .orderBy("zona_ocu")

df_resumen.show()


In [0]:
# Convertimos a pandas
pdf = df_resumen.toPandas()

import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12,6))
sns.barplot(x="zona_ocu", y="total_accidentes", data=pdf, palette="Blues_d")
plt.xlabel("Zona")
plt.ylabel("Total de Accidentes")
plt.title("Accidentes por Zona - Municipio de Guatemala")
plt.xticks(rotation=45)
plt.show()


Podemos ver que la mayoia de accidentes suceden en la zona 7 y en zona 12. Esto muy posible porque son zonas no de viviendo sino en donde estan las carreteras mas importantes de la ciudad, esto causa mas congestion y probabilidad que ocurran accidentes.

**Crear un DataFrame que muestre el porcentaje de accidentes donde el conductor
era hombre vs mujer (tabla vehículos). Guardar como Parquet. Finalmente, vuélvalo a
cargar y grafique con display en gráfico de pie**

In [0]:
from pyspark.sql.functions import col, count, round as spark_round

# Cargar tabla vehiculos
df_veh = spark.table("vehiculos")

# Contar por sexo
df_sexo = df_veh.groupBy("sexo").agg(count("*").alias("total"))

# Calcular porcentaje usando round de pyspark
total_accidentes = df_veh.count()
df_sexo = df_sexo.withColumn(
    "porcentaje",
    spark_round(col("total") / total_accidentes * 100, 2)
)

# Reemplazar valores de sexo por nombre usando int en lugar de float
df_sexo = df_sexo.replace({1: "Hombre", 2: "Mujer", 9: "Ignorado"}, "sexo")

df_sexo.show()


In [0]:
from pyspark.sql.functions import col, count, round as spark_round, when

# Contar por sexo
df_sexo = df_veh.groupBy("sexo").agg(count("*").alias("total"))

# Calcular porcentaje
total_accidentes = df_veh.count()
df_sexo = df_sexo.withColumn(
    "porcentaje",
    spark_round(col("total") / total_accidentes * 100, 2)
)

# Reemplazar usando when / otherwise
df_sexo = df_sexo.withColumn(
    "sexo",
    when(col("sexo") == 1.0, "Hombre")
    .when(col("sexo") == 2.0, "Mujer")
    .when(col("sexo") == 9.0, "Ignorado")
    .otherwise("Desconocido")
)

df_sexo.show()


In [0]:
# Path local para guardar Parquet
ruta_parquet_local = "/Volumes/workspace/default/lab8_new/vehiculos_por_sexo.parquet"

df_sexo.write.mode("overwrite").parquet(ruta_parquet_local)

df_sexo_recargado = spark.read.parquet(ruta_parquet_local)

display(df_sexo_recargado)


In [0]:
df_sexo_pd = df_sexo_recargado.toPandas()

# Datos para el gráfico
labels = df_sexo_pd["sexo"]
sizes = df_sexo_pd["total"]

# Colores opcionales
colors = ["#1f77b4", "#ff7f0e", "#2ca02c"]  # Hombre, Mujer, Ignorado

# Graficar
plt.figure(figsize=(7,7))
plt.pie(sizes, labels=labels, autopct="%1.1f%%", colors=colors, startangle=140)
plt.title("Distribución de accidentes por sexo")
plt.show()

Como podemos ver los hombres son los que mas accidentes de vehiculos causan a las personas.Esto puede ser debido a que son el sector de la poblacion que mas conduce y debido a esto generan muchos accidentes. 

Y el menor cantidad de accidentes causados es los hombres. 